<a href="https://colab.research.google.com/github/Hanbin-git/kaggle/blob/main/RNA2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

stanford_rna_3d_folding_path = kagglehub.competition_download('stanford-rna-3d-folding')
metric_usalign_path = kagglehub.dataset_download('metric/usalign')
geraseva_protenix_checkpoints_path = kagglehub.dataset_download('geraseva/protenix-checkpoints')
metric_ribonanza_tm_score_path = kagglehub.notebook_output_download('metric/ribonanza-tm-score')

print('Data source import complete.')


In [ ]:
MODEL_TYPE='protenix'
VALIDATION=False

In [ ]:
!pip uninstall -y torch torchvision torchaudio protenix

Found existing installation: torch 2.1.2+cu118
Uninstalling torch-2.1.2+cu118:
  Successfully uninstalled torch-2.1.2+cu118
Found existing installation: protenix 0.4.6
Uninstalling protenix-0.4.6:
  Successfully uninstalled protenix-0.4.6


In [ ]:
!pip install --extra-index-url https://download.pytorch.org/whl/cu118 torch==2.1.2+cu118

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu118
  Using cached https://download.pytorch.org/whl/cu118/torch-2.1.2%2Bcu118-cp310-cp310-linux_x86_64.whl (2325.9 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
easyocr 1.7.2 requires torchvision>=0.5, which is not installed.
fastai 2.7.18 requires torchvision>=0.11, which is not installed.
timm 1.0.12 requires torchvision, which is not installed.


In [ ]:
!pip install --no-deps protenix==0.4.6 einops tqdm gemmi biotite==1.0.1

  Using cached protenix-0.4.6-py3-none-any.whl.metadata (1.1 kB)
Using cached protenix-0.4.6-py3-none-any.whl (441 kB)


## Install requirements

In [ ]:
if MODEL_TYPE=='protenix' and VALIDATION:
    !pip install biopython
    !pip install ml-collections
    !pip install rdkit
!export PROTENIX_DATA_ROOT_DIR=/kaggle/input/protenix-checkpoints

In [ ]:
! mkdir /af3-dev
! ln -s /kaggle/input/protenix-checkpoints /af3-dev/release_data
! ls /af3-dev/release_data/

mkdir: cannot create directory ‘/af3-dev’: File exists
ln: failed to create symbolic link '/af3-dev/release_data/protenix-checkpoints': Read-only file system
components.v20240608.cif		model_v0.2.0.pt
components.v20240608.cif.rdkit_mol.pkl


## Helper scripts

In [ ]:
import Bio

from copy import deepcopy

import pandas as pd
from Bio.PDB import Atom, Model, Chain, Residue, Structure, PDBParser
from Bio import SeqIO
import os, sys
import re
import numpy as np
import torch

import matplotlib
import matplotlib.pyplot as plt
from tqdm import tqdm
import time
time0=time.time()

print('IMPORT OK !!!!')

IMPORT OK !!!!


In [ ]:

PYTHON = sys.executable
print('PYTHON',PYTHON)

RHONET_DIR=\
'/kaggle/input/data-for-demo-for-rhofold-plus-with-kaggle-msa/RhoFold-main'
#'<your downloaded rhofold repo>/RhoFold-main'

USALIGN = \
'/kaggle/working//USalign'
#'<your us align path>/USalign'

os.system('cp /kaggle/input/usalign/USalign /kaggle/working/')
os.system('sudo chmod u+x /kaggle/working//USalign')
sys.path.append(RHONET_DIR)


DATA_KAGGLE_DIR = '/kaggle/input/stanford-rna-3d-folding'


# helper ----
class dotdict(dict):
	__setattr__ = dict.__setitem__
	__delattr__ = dict.__delitem__

	def __getattr__(self, name):
		try:
			return self[name]
		except KeyError:
			raise AttributeError(name)

# visualisation helper ----
def set_aspect_equal(ax):
	x_limits = ax.get_xlim()
	y_limits = ax.get_ylim()
	z_limits = ax.get_zlim()

	# Compute the mean of each axis
	x_middle = np.mean(x_limits)
	y_middle = np.mean(y_limits)
	z_middle = np.mean(z_limits)

	# Compute the max range across all axes
	max_range = max(x_limits[1] - x_limits[0],
					y_limits[1] - y_limits[0],
					z_limits[1] - z_limits[0]) / 2.0

	# Set the new limits to ensure equal scaling
	ax.set_xlim(x_middle - max_range, x_middle + max_range)
	ax.set_ylim(y_middle - max_range, y_middle + max_range)
	ax.set_zlim(z_middle - max_range, z_middle + max_range)


PYTHON /usr/bin/python3


In [ ]:
# xyz df helper --------------------
def get_truth_df(target_id):
    truth_df = LABEL_DF[LABEL_DF['target_id'] == target_id]
    truth_df = truth_df.reset_index(drop=True)
    return truth_df

def parse_output_to_df(output, seq, target_id):
    df = []
    chain_data = []
    for i, res in enumerate(seq):
        d=dict(ID = target_id,
                    resname=res,
                    resid=i+1)
        for n in range(len(output)):
            d={**d, f'x_{n+1}': round(output[n,i,0].item(),3),
                     f'y_{n+1}': round(output[n,i,1].item(),3),
                     f'z_{n+1}': round(output[n,i,2].item(),3)}
        chain_data.append(d)

    if len(chain_data)!=0:
        chain_df = pd.DataFrame(chain_data)
        df.append(chain_df)
        ##print(chain_df)
    return df

def parse_pdb_to_df(pdb_file, target_id):
    parser = PDBParser()
    structure = parser.get_structure('', pdb_file)

    df = []
    for model in structure:
        for chain in model:
            print(chain)
            chain_data = []
            for residue in chain:
                # print(residue)
                if residue.get_resname() in ['A', 'U', 'G', 'C']:
                    # Check if the residue has a C1' atom
                    if 'C1\'' in residue:
                        atom = residue['C1\'']
                        xyz = atom.get_coord()
                        resname = residue.get_resname()
                        resid = residue.get_id()[1]

                        #todo detect discontinous: resid = prev_resid+1
                        #ID	resname	resid	x_1	y_1	z_1
                        chain_data.append(dict(
                            ID = target_id+'_'+str(resid),
                            resname=resname,
                            resid=resid,
                            x_1=xyz[0],
                            y_1=xyz[1],
                            z_1=xyz[2],
                        ))
                        ##print(f"Residue {resname} {resid}, Atom: {atom.get_name()}, xyz: {xyz}")

            if len(chain_data)!=0:
                chain_df = pd.DataFrame(chain_data)
                df.append(chain_df)
                ##print(chain_df)
    return df

In [ ]:

# usalign helper --------------------
def write_target_line(
    atom_name, atom_serial, residue_name, chain_id, residue_num, x_coord, y_coord, z_coord, occupancy=1.0, b_factor=0.0, atom_type='P'
):
    """
    Writes a single line of PDB format based on provided atom information.

    Args:
        atom_name (str): Name of the atom (e.g., "N", "CA").
        atom_serial (int): Atom serial number.
        residue_name (str): Residue name (e.g., "ALA").
        chain_id (str): Chain identifier.
        residue_num (int): Residue number.
        x_coord (float): X coordinate.
        y_coord (float): Y coordinate.
        z_coord (float): Z coordinate.
        occupancy (float, optional): Occupancy value (default: 1.0).
        b_factor (float, optional): B-factor value (default: 0.0).

    Returns:
        str: A single line of PDB string.
    """
    return f'ATOM  {atom_serial:>5d}  {atom_name:<5s} {residue_name:<3s} {residue_num:>3d}    {x_coord:>8.3f}{y_coord:>8.3f}{z_coord:>8.3f}{occupancy:>6.2f}{b_factor:>6.2f}           {atom_type}\n'

def write_xyz_to_pdb(df, pdb_file, xyz_id = 1):
    resolved_cnt = 0
    with open(pdb_file, 'w') as target_file:
        for _, row in df.iterrows():
            x_coord = row[f'x_{xyz_id}']
            y_coord = row[f'y_{xyz_id}']
            z_coord = row[f'z_{xyz_id}']

            if x_coord > -1e17 and y_coord > -1e17 and z_coord > -1e17:
                resolved_cnt += 1
                target_line = write_target_line(
                    atom_name="C1'",
                    atom_serial=int(row['resid']),
                    residue_name=row['resname'],
                    chain_id='0',
                    residue_num=int(row['resid']),
                    x_coord=x_coord,
                    y_coord=y_coord,
                    z_coord=z_coord,
                    atom_type='C',
                )
                target_file.write(target_line)
    return resolved_cnt

def parse_usalign_for_tm_score(output):
    # Extract TM-score based on length of reference structure (second)
    tm_score_match = re.findall(r'TM-score=\s+([\d.]+)', output)[1]
    if not tm_score_match:
        raise ValueError('No TM score found')
    return float(tm_score_match)

def parse_usalign_for_transform(output):
    # Locate the rotation matrix section
    matrix_lines = []
    found_matrix = False

    for line in output.splitlines():
        if "The rotation matrix to rotate Structure_1 to Structure_2" in line:
            found_matrix = True
        elif found_matrix and re.match(r'^\d+\s+[-\d.]+\s+[-\d.]+\s+[-\d.]+\s+[-\d.]+$', line):
            matrix_lines.append(line)
        elif found_matrix and not line.strip():
            break  # Stop parsing if an empty line is encountered after the matrix

    # Parse the rotation matrix values
    rotation_matrix = []
    for line in matrix_lines:
        parts = line.split()
        row_values = list(map(float, parts[1:]))  # Skip the first column (index)
        rotation_matrix.append(row_values)

    return np.array(rotation_matrix)

def call_usalign(predict_df, truth_df, verbose=1):
    truth_pdb = '~truth.pdb'
    predict_pdb = '~predict.pdb'
    write_xyz_to_pdb(predict_df, predict_pdb, xyz_id=1)
    write_xyz_to_pdb(truth_df, truth_pdb, xyz_id=1)

    command = f'{USALIGN} {predict_pdb} {truth_pdb} -atom " C1\'" -m -'
    output = os.popen(command).read()
    if verbose==1:
        print(output)
    tm_score = parse_usalign_for_tm_score(output)
    transform = parse_usalign_for_transform(output)
    return tm_score, transform

print('HELPER OK!!!')

HELPER OK!!!


In [ ]:

if MODEL_TYPE=='protenix':


    from runner.batch_inference import get_default_runner
    from runner.inference import update_inference_configs, InferenceRunner

    from protenix.data.infer_data_pipeline import InferenceDataset

    np.random.seed(0)
    torch.random.manual_seed(0)
    torch.cuda.manual_seed_all(0)

    class DictDataset(InferenceDataset):
        def __init__(
            self,
            seq_list: list,
            dump_dir: str,
            id_list: list = None,
            use_msa: bool = False,
        ) -> None:

            self.dump_dir = dump_dir
            self.use_msa = use_msa
            if isinstance(id_list,type(None)):
                self.inputs = [{"sequences":
                                [{"rnaSequence":
                                  {"sequence": seq,
                                   "count": 1}}],
                                "name": "query"} for seq in seq_list]
            else:
                self.inputs = [{"sequences":
                                [{"rnaSequence":
                                  {"sequence": seq,
                                   "count": 1}}],
                                "name": i} for i, seq in zip(id_list,seq_list)]


In [ ]:
def smooth_coords(coord_tensor, window=3):
    coords = coord_tensor.squeeze().cpu().numpy()
    smoothed = np.copy(coords)
    for j in range(1, len(coords) - 1):
        smoothed[j] = np.mean(coords[j - 1:j + 2], axis=0)
    return torch.tensor(smoothed).unsqueeze(0).to(coord_tensor.device)

if MODEL_TYPE == 'protenix' and not VALIDATION:
    import warnings
    warnings.filterwarnings("ignore")
    test_df = pd.read_csv('/kaggle/input/stanford-rna-3d-folding/test_sequences.csv')

    dataset = DictDataset(test_df.sequence, dump_dir='output', id_list=test_df.target_id, use_msa=False)
    num_data = len(dataset)

    for i, seq in tqdm(enumerate(test_df.sequence), total=num_data):
        try:
            data, atom_array, data_error_message = dataset[i]
            target_id = data["sample_name"]

            if data_error_message != '':
                print(f"[!] Skipped: {target_id} ▶ data error: {data_error_message}")
                raise ValueError("Invalid input features")

            # config 업데이트
            new_configs = update_inference_configs(configs, data["N_token"].item())
            new_configs["eval_dtype"] = "float16"
            if "model" not in new_configs:
                new_configs["model"] = {}
            new_configs["model"]["eval_dtype"] = "float16"
            runner.update_model_configs(new_configs)

            prediction = runner.predict(data)
            coords = prediction['coordinate'][:, data['input_feature_dict']['atom_to_tokatom_idx'] == 12]

            if coords.shape[1] != len(seq):
                print(f"[!] Shape mismatch: {target_id} ▶ coords: {coords.shape[1]}, seq: {len(seq)}")
                raise ValueError("Output shape mismatch")

            # smoothing 적용
            coords = smooth_coords(coords)

            result = parse_output_to_df(coords, seq, target_id)[0]

        except Exception as e:
            print(f"[!] Failed to predict {target_id} ▶ {e}")
            result = pd.DataFrame(columns=['ID', 'resname', 'resid',
                                           'x_1', 'y_1', 'z_1',
                                           'x_2', 'y_2', 'z_2',
                                           'x_3', 'y_3', 'z_3',
                                           'x_4', 'y_4', 'z_4',
                                           'x_5', 'y_5', 'z_5'],
                                 data=[[target_id, x, j+1] + [0.0]*15 for j, x in enumerate(seq)])

        result['ID'] = result.apply(lambda x: f"{x.ID}_{x.resid}", axis=1)
        result.to_csv('submission.csv', index=False, mode='a', header=(i == 0), float_format="%.3f")
        torch.cuda.empty_cache()

    display(pd.read_csv('submission.csv'))
